In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#Import basic libraries
%matplotlib inline


import os
import sys
import settings
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from django_pandas.io import read_frame
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

from matplotlib.dates import date2num

In [3]:
#Prepare to load stock data as pandas dataframe from source. In this case, prepare django
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from stocks.models import Listing, Stock, Market

In [4]:
#Asian paints
stock = Stock.objects.get(sid='ASIANPAINT') #Load Nifty

print (stock.sid)
print(stock.name)
print (stock)


ASIANPAINT
ASIAN PAINTS LTD.
ASIAN PAINTS LTD.


In [5]:
#Read data into a dataframe
listings = Listing.objects.filter(stock=stock)
df = read_frame(listings, index_col='date')
for column in df.columns:
    if column != 'stock':
       df[column] = pd.to_numeric(df[column]) 
df = df.sort_index()
df = df.reindex(columns = ['opening', 'high', 'low', 'closing', 'traded'])
df.rename(columns={"opening": "Open", "high": "High", "low": "Low", "closing":"Close", "traded":"Volume"}, inplace=True)

In [6]:
#Optionally, filter out by date range
start_date = '2020-01-01'
end_date = '2021-12-31'
ticker = 'ASIANPAINTS'

df = df.loc[start_date:end_date]

In [7]:
df.head()

,Open,High,Low,Close,Volume
date,,,,,
2020-01-01,1790.15,1803.0,1785.00,1793.75,32156
2020-01-02,1790.60,1800.0,1785.00,1790.25,22940
2020-01-03,1788.00,1788.0,1747.25,1751.65,26202
2020-01-06,1744.90,1744.9,1694.00,1707.50,68842
2020-01-07,1708.30,1740.8,1708.30,1723.90,20856


In [8]:
#Import TA-lib and backtesting library
import talib
from talib.abstract import *
from talib import MA_Type

from backtesting import Strategy
from backtesting.lib import crossover

/home/anshul/web/experimental/env/lib/python3.6/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
#Create a strategy
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 20
    n2 = 200
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, int(self.n1))
        self.sma2 = self.I(SMA, self.data.Close, int(self.n2))
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
from backtesting import Backtest

bt = Backtest(df, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

In [ ]:
bt.plot()

In [ ]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(20, 200, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

In [ ]:
stats._strategy

In [ ]:
bt.plot(plot_volume=False, plot_pl=False)

In [ ]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, int(self.n1))
        sma2 = self.I(SMA, self.data.Close, int(self.n2))
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [ ]:
bt = Backtest(df, SmaCross, commission=.002)
bt.run()
bt.plot()

In [ ]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(20, 200, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

In [ ]:
stats._strategy

In [ ]:
bt.plot(plot_volume=False, plot_pl=False)

In [ ]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 14  # Daily RSI lookback periods
    w_rsi = 14  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, int(10))
        self.ma20 = self.I(SMA, self.data.Close, int(20))
        self.ma50 = self.I(SMA, self.data.Close, int(50))
        self.ma100 = self.I(SMA, self.data.Close, int(100))
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(talib.RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', talib.RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [ ]:
backtest = Backtest(df, System, commission=.002)
backtest.run()

In [ ]:
backtest.plot()

In [ ]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

In [ ]:
backtest.plot()

 `argrelextrema` is used for detecting peaks in SciPy's signal processing library, and `deque` is like a fixed-length list that will drop the oldest entry and keep the new ones if you exceed its length. We'll use the first to spot our extrema in our data, then cycle through them and keep the points that are higher than the previous entries.

To spot our extrema, we need to pass an argument called order. This defines how many points on either side of our peak we need to actually label something a peak. So with order=5, we need something to be the highest point within 5-data points to the right and left. The other argument we provide is K, which is simply an integer to determine how many consecutive peaks we want to identify to determine a trend of higher highs.

In [ ]:
#RSI Divergence strategy
#https://raposa.trade/blog/test-and-trade-rsi-divergence-in-python

from scipy.signal import argrelextrema
from collections import deque

def getHigherLows(data: np.array, order=5, K=2):
  '''
  Finds consecutive higher lows in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive lows need to be higher.
  '''
  # Get lows
  low_idx = argrelextrema(data, np.less, order=order)[0]
  lows = data[low_idx]
  # Ensure consecutive lows are higher than previous lows
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(low_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if lows[i] < lows[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getLowerHighs(data: np.array, order=5, K=2):
  '''
  Finds consecutive lower highs in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive highs need to be lower.
  '''
  # Get highs
  high_idx = argrelextrema(data, np.greater, order=order)[0]
  highs = data[high_idx]
  # Ensure consecutive highs are lower than previous highs
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(high_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if highs[i] > highs[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getHigherHighs(data: np.array, order=5, K=2):
  '''
  Finds consecutive higher highs in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive highs need to be higher.
  '''
  # Get highs
  high_idx = argrelextrema(data, np.greater, order=5)[0]
  highs = data[high_idx]
  # Ensure consecutive highs are higher than previous highs
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(high_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if highs[i] < highs[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getLowerLows(data: np.array, order=5, K=2):
  '''
  Finds consecutive lower lows in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive lows need to be lower.
  '''
  # Get lows
  low_idx = argrelextrema(data, np.less, order=order)[0]
  lows = data[low_idx]
  # Ensure consecutive lows are lower than previous lows
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(low_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if lows[i] > lows[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema


In [ ]:
from matplotlib.lines import Line2D # For legend
price = df['Close'].values
dates = df.index
# Get higher highs, lower lows, etc.
order = 5
hh = getHigherHighs(price, order)
lh = getLowerHighs(price, order)
ll = getLowerLows(price, order)
hl = getHigherLows(price, order)
# Get confirmation indices
hh_idx = np.array([i[1] + order for i in hh])
lh_idx = np.array([i[1] + order for i in lh])
ll_idx = np.array([i[1] + order for i in ll])
hl_idx = np.array([i[1] + order for i in hl])
# Plot results
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.figure(figsize=(12, 8))
plt.plot(df['Close'])
plt.scatter(dates[hh_idx], price[hh_idx-order], marker='^', c=colors[1])
plt.scatter(dates[lh_idx], price[lh_idx-order], marker='v', c=colors[2])
plt.scatter(dates[ll_idx], price[ll_idx-order], marker='v', c=colors[3])
plt.scatter(dates[hl_idx], price[hl_idx-order], marker='^', c=colors[4])
_ = [plt.plot(dates[i], price[i], c=colors[1]) for i in hh]
_ = [plt.plot(dates[i], price[i], c=colors[2]) for i in lh]
_ = [plt.plot(dates[i], price[i], c=colors[3]) for i in ll]
_ = [plt.plot(dates[i], price[i], c=colors[4]) for i in hl]
plt.xlabel('Date')
plt.ylabel('Price (Rs)')
plt.title(f'Potential Divergence Points for {ticker} Closing Price')
legend_elements = [
  Line2D([0], [0], color=colors[0], label='Close'),
  Line2D([0], [0], color=colors[1], label='Higher Highs'),
  Line2D([0], [0], color='w',  marker='^',
         markersize=10,
         markerfacecolor=colors[1],
         label='Higher High Confirmation'),
  Line2D([0], [0], color=colors[2], label='Higher Lows'),
  Line2D([0], [0], color='w',  marker='^',
         markersize=10,
         markerfacecolor=colors[2],
         label='Higher Lows Confirmation'),
  Line2D([0], [0], color=colors[3], label='Lower Lows'),
  Line2D([0], [0], color='w',  marker='v',
         markersize=10,
         markerfacecolor=colors[3],
         label='Lower Lows Confirmation'),
  Line2D([0], [0], color=colors[4], label='Lower Highs'),
  Line2D([0], [0], color='w',  marker='^',
         markersize=10,
         markerfacecolor=colors[4],
         label='Lower Highs Confirmation')
]
plt.legend(handles=legend_elements, bbox_to_anchor=(1, 0.65))
plt.show()

In [ ]:
#Compute the RSI of the data
rsi_duration = 14;
rsi = talib.RSI(df['Close'].values, rsi_duration)
df['RSI'] =  rsi

rsi_hh = getHigherHighs(rsi, order)
rsi_lh = getLowerHighs(rsi, order)
rsi_ll = getLowerLows(rsi, order)
rsi_hl = getHigherLows(rsi, order)

# Get confirmation indices
rsi_hh_idx = np.array([i[1] + order for i in rsi_hh])
rsi_lh_idx = np.array([i[1] + order for i in rsi_lh])
rsi_ll_idx = np.array([i[1] + order for i in rsi_ll])
rsi_hl_idx = np.array([i[1] + order for i in rsi_hl])

In [ ]:
fig, ax = plt.subplots(2, figsize=(20, 12), sharex=True)
ax[0].plot(df['Close'])
ax[0].scatter(dates[hh_idx], price[hh_idx-order], 
              marker='^', c=colors[1])
ax[0].scatter(dates[lh_idx], price[lh_idx-order],
              marker='v', c=colors[2])
ax[0].scatter(dates[hl_idx], price[hl_idx-order],
              marker='^', c=colors[3])
ax[0].scatter(dates[ll_idx], price[ll_idx-order],
              marker='v', c=colors[4])
_ = [ax[0].plot(dates[i], price[i], c=colors[1]) for i in hh]
_ = [ax[0].plot(dates[i], price[i], c=colors[2]) for i in lh]
_ = [ax[0].plot(dates[i], price[i], c=colors[3]) for i in hl]
_ = [ax[0].plot(dates[i], price[i], c=colors[4]) for i in ll]
ax[0].set_ylabel('Price (Rs)')
ax[0].set_title(f'Price and Potential Divergence Points for {ticker}')
ax[0].legend(handles=legend_elements)
ax[1].plot(df['RSI'])
ax[1].scatter(dates[rsi_hh_idx], rsi[rsi_hh_idx-order], 
              marker='^', c=colors[1])
ax[1].scatter(dates[rsi_lh_idx], rsi[rsi_lh_idx-order],
              marker='v', c=colors[2])
ax[1].scatter(dates[rsi_hl_idx], rsi[rsi_hl_idx-order],
              marker='^', c=colors[3])
ax[1].scatter(dates[rsi_ll_idx], rsi[rsi_ll_idx-order],
              marker='v', c=colors[4])
_ = [ax[1].plot(dates[i], rsi[i], c=colors[1]) for i in rsi_hh]
_ = [ax[1].plot(dates[i], rsi[i], c=colors[2]) for i in rsi_lh]
_ = [ax[1].plot(dates[i], rsi[i], c=colors[3]) for i in rsi_hl]
_ = [ax[1].plot(dates[i], rsi[i], c=colors[4]) for i in rsi_ll]
ax[1].set_ylabel('RSI')
ax[1].set_title(f'RSI and Potential Divergence Points for {ticker}')
ax[1].set_xlabel('Date')
plt.tight_layout()
plt.show() 

In [ ]:
#RSI Divergence strategy
def getHHIndex(data: np.array, order=5, K=2):
  extrema = getHigherHighs(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getLHIndex(data: np.array, order=5, K=2):
  extrema = getLowerHighs(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getLLIndex(data: np.array, order=5, K=2):
  extrema = getLowerLows(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getHLIndex(data: np.array, order=5, K=2):
  extrema = getHigherLows(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getPeaks(data, key='Close', order=5, K=2):
  vals = data[key].values
  hh_idx = getHHIndex(vals, order, K)
  lh_idx = getLHIndex(vals, order, K)
  ll_idx = getLLIndex(vals, order, K)
  hl_idx = getHLIndex(vals, order, K)

  data[f'{key}_highs'] = np.nan
  data[f'{key}_highs'][hh_idx] = 1
  data[f'{key}_highs'][lh_idx] = -1
  data[f'{key}_highs'] = data[f'{key}_highs'].ffill().fillna(0)
  data[f'{key}_lows'] = np.nan
  data[f'{key}_lows'][ll_idx] = 1
  data[f'{key}_lows'][hl_idx] = -1
  data[f'{key}_lows'] = data[f'{key}_highs'].ffill().fillna(0)
  return data


In [ ]:
from backtesting import Strategy
def getPeaks(data, key='Close', order=5, K=2):
    
    vals = data[key].values
    hh_idx = getHHIndex(vals, order, K)
    lh_idx = getLHIndex(vals, order, K)
    
    a = np.empty(vals.shape)
    a[:] = np.nan
    a[hh_idx] = 1
    a[lh_idx] = -1
    #a = a.ffill().fillna(0)
    a[np.isnan(a)] = 0
    return a

def getValleys(data, key='Close', order=5, K=2):
    vals = data[key].values
    ll_idx = getLLIndex(vals, order, K)
    hl_idx = getHLIndex(vals, order, K)
    
    a = np.empty(vals.shape)
    a[:] = np.nan
    a[ll_idx] = 1
    a[hl_idx] = -1
    #a = data[f'{key}_highs'].ffill().fillna(0)
    a[np.isnan(a)] = 0
    return a
    
#Create a strategy
class RsiDivergence(Strategy):
    '''
      Go long/short on price and RSI divergence.
      - Long if price to lower low and RSI to higher low with RSI < 50
      - Short if price to higher high and RSI to lower high with RSI > 50
      Sell if divergence disappears.
      Sell if the RSI crosses the centerline.
    '''
    rsi_level = 50
    order = 5
    K = 2
    rsi_period = 14
    
    ema_s = 50
    ema_l = 200
    
    
    def init(self):
        # Precompute the RSI
        self.ema_s = self.I(EMA, self.data.df['Close'], int(self.ema_s))
        self.ema_l = self.I(EMA, self.data.df['Close'], int(self.ema_l))
        self.data.df['RSI'] = talib.RSI(self.data.df['Close'], self.rsi_period)
        self.rsi = self.I(talib.RSI, self.data.df['Close'], self.rsi_period)
        
        self.entry_rsi = 0
        
        self.close_highs = self.I(getPeaks, self.data.df, key='Close', order=self.order, K=self.K)
        self.close_lows = self.I(getValleys, self.data.df, key='Close', order=self.order, K=self.K)
        
        self.rsi_highs = self.I(getPeaks, self.data.df, key='RSI', order=self.order, K=self.K)
        self.rsi_lows = self.I(getValleys, self.data.df, key='RSI', order=self.order, K=self.K)
    
    def next(self):
        price = self.data.Close[-1]
        if np.isnan(self.data.RSI[-1]):
            return
        
        if self.position is None: #We might be opening a position
            #Long if RSI is less than 50 and making higher high
            if self.close_lows[-1] == -1 and self.rsi_lows == 1: #Price makes lower low, indicator makes higher low (Positive divergence)
                if self.data.RSI[-1] < 50:
                    self.entry_rsi = self.data.RSI[-1].copy()
                    #self.buy(sl=.92 * price)
                    self.buy()       
            # Short if price to higher high and indicator to lower high
            elif self.close_highs[-1] == 1 and self.rsi_highs[-1] == -1: #Price making higher high, RSI making lower high (Negative divergence)
                if row['RSI'] > 50:
                    self.sell()
                    self.entry_rsi = self.data.RSI[-1].copy()
        # If current position is long
        elif self.position.is_long:
            if self.rsi[-1] < 50 and self.rsi[-1] < self.entry_rsi:
                pass
            else:
                self.position.close()
            #elif self.ema_s > self.ema_l:
            #    pass

        # If current position is short
        elif self.position.is_short:
            if self.rsi[-1] > 50 and self.rsi[-1] < self.entry_rsi:
                pass
            else:
                self.position.close()
                

In [ ]:
from backtesting import Backtest

bt = Backtest(df, RsiDivergence, cash=10_000, commission=.002)
stats = bt.run()
stats


In [ ]:
bt.plot()

## Bollinger band and RSI strategy

In [ ]:
#https://kylelix7.github.io/Trading-Strategy-Technical-Analysis-with-Python-TA-Lib/
#Create a strategy
class BollingerBand(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    timeperiod = 20
    rsi_period = 14
    nbdevup = 2
    nbdevdn = 2
    
    def init(self):
        # Precompute the two moving averages
        self.rsi = self.I(talib.RSI, self.data.Close, int(self.rsi_period))
        self.up, self.mid, self.low = self.I(talib.BBANDS, self.data.Close, 
                                              timeperiod = int(self.timeperiod), 
                                              nbdevup = int(self.nbdevup),
                                             nbdevdn = int(self.nbdevdn),
                                             matype=0)
    
    def next(self):
        bbp = (self.data.Close[-1] - self.low[-1]) / (self.up[-1] - self.low[-1])
        if self.rsi[-1]>70 and bbp>1:
            self.position.close()
            self.sell()

        if self.rsi[-1]<30 and bbp<0:
            self.position.close()
            self.buy()

In [ ]:
from backtesting import Backtest

bt = Backtest(df, BollingerBand, cash=10_000, commission=.002)
stats = bt.run()
stats

## EMA Trend Following Scalping strategy on intraday

In [29]:
#Load Nifty data
df = pd.read_csv('./NIFTY50.csv', index_col=0)
#df = pd.read_csv('./NIFTY50.csv')
for column in df.columns:
    if column != 'date':
       df[column] = pd.to_numeric(df[column])
#df['date'] = pd.to_datetime(df['date'])
#df['date'] = df['date'].tz_localize(None)
#df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize(None)

df = df.sort_index()
df = df.reindex(columns = ['open', 'high', 'low', 'close', 'volume'])
df.drop('volume', axis=1, inplace=True)
df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close":"close"}, inplace=True)

df = df[~df.index.duplicated(keep='first')]
#Optionally, filter out by date range
#start_date = '2020-01-01'
#end_date = '2021-12-31'
#ticker = 'ASIANPAINTS'

#df = df.loc[start_date:end_date]
#drop duplicates


In [30]:
#print(df.index.dtype)
print(df.head())

datetime64[ns]
                        Open     High      Low    close
date                                                   
2015-01-09 09:15:00  8285.45  8295.90  8285.45  8292.10
2015-01-09 09:16:00  8292.60  8293.60  8287.20  8288.15
2015-01-09 09:17:00  8287.40  8293.90  8287.40  8293.90
2015-01-09 09:18:00  8294.25  8300.65  8293.90  8300.65
2015-01-09 09:19:00  8300.60  8301.30  8298.75  8301.20


In [ ]:
class EMAStrategy(Strategy):
    '''
    If the price is below the EMA, trend is down, else up.
    
    If price touches EMA line for the first time, take note of trend (might be retest, might be trend change)
    
    '''
    EMA_period = 10
    
    def init(self):
        # Precompute the moving average
        self.ema = self.I(talib.EMA, self.data.Close, int(self.EMA_period))
        
        self.direction = 0
        self.confirm = 0
        self.stoploss = 0
    
    def next(self):
        #Has the price crossed over?
        if crossover(self.data.Close, self.ema):
            if self.direction == -1 and self.confirm == 0: #Could be confirmation
                pass
            elif self.direction <= 0 and self.confirm == 0: #Might break to upside
                self.direction = 1
                self.confirm = 0
                #self.position.close() #Close any bearish positions
            elif self.direction > 0 and self.confirm == 0: #Confirmed uptrend
                self.confirm = 1
                self.buy()
            elif self.direction > 0 and self.confirm == 1: #Retest
                self.buy() #Add more or hold
            elif self.direction < 0 and self.confirm == 1: #Close bearish position now
                self.position.close()
        elif crossover(self.ema, self.data.Close):
            if self.direction == 1 and self.confirm == 0: #Could be confirmation
                pass
            elif self.direction <= 0 and self.confirm == 0: #Might break to downside
                self.direction = -1
                self.confirm = 0
                #self.position.close() #Close any bullish positions
            elif self.direction < 0 and self.confirm == 0: #Confirmed downtrend
                self.confirm = 1
                self.sell()
            elif self.direction < 0 and self.confirm == 1: #Retest
                self.sell() #Short more or hold
            elif self.direction > 0 and self.confirm == 1: #Close bullish position now
                self.position.close()
            


In [ ]:
from backtesting import Backtest

bt = Backtest(df, EMAStrategy, cash=10_000, commission=.002)
stats = bt.run()
stats